In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
def Normalization(x):
    max_val = max(x)
    min_val = min(x)
    return [(i-min_val)/(max_val-min_val) for i in x]

In [3]:
def weight_lost_cal(real, pre):
    
    test_auc = metrics.roc_auc_score(real, pre)
    fpr, tpr, thresholds = metrics.roc_curve(real, pre, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    
    print("AUC value = " + str(test_auc))
    print("Score = " + str(score))
    return score

In [4]:
# read data
root = "./data/"
data_train = pd.read_csv(root + "atec_anti_fraud_train.csv")
# data_train_add = pd.read_csv(root + "predicted_positive_sample_test_add.csv")
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")

In [5]:
# data preprocessing
# 是否要采样
# data_train = data_train.sample(100000)
# data_train = data_train[data_train['label'] != -1]

data_test = data_test.drop(['id'], 1)
data_test = data_test.drop(['date'], 1)

data_train = data_train.drop(['id'], 1)
data_train = data_train.drop(['date'], 1)
# data_train_add = data_train_add.drop(['id'], 1)
# data_train_add = data_train_add.drop(['date'], 1)
# data_train = pd.concat([data_train,data_train_add], sort=True)

data_train['label'] = np.abs(data_train['label'])
train_y = data_train['label']
data_train = data_train.drop(['label'], 1)

# 删除特征
b = ['20','22','24','26','28','30','32','34','46','47','48','50','52','53','100','13','49','109','107','104','102','161','201']        
c = [str(i) for i in range(64,72)]
d = [str(i) for i in range(111,155)]

drop_col = ['f'+i for i in (b+c+d)]

data_test = data_test.drop(drop_col,1)
data_train = data_train.drop(drop_col,1)

In [6]:
train_x = data_train
test_x = data_test
# train_y = np.array(train_y).reshape(-1, 1)

# train_x = np.array(train_x)
# train_y = np.array(train_y).reshape(-1, 1)
# test_x = np.array(test_x)

X = np.array(train_x)
Y = np.array(train_y)
X_test = test_x

In [7]:
X_test = test_x

In [8]:
# train
# 交叉验证取平均
k = 5
kf = KFold(n_splits = k, shuffle=True, random_state=2)
# kf = KFold(n_splits=k)

eta = 0.05
result = 0
xgb_result = 0
importances = 0
for train_index, test_index in kf.split(X):
    X1_train, X1_test = X[train_index], X[test_index]
    Y1_train, Y1_test = Y[train_index], Y[test_index]

    data_train = lgb.Dataset(X1_train, Y1_train)
    data_vtest = lgb.Dataset(X1_test, Y1_test)
#     data_rtest = lgb.Dataset(X_test)
    param = {
        'boosting_type': 'gbdt',
        'objective': 'binary',  # 二分类问题
        # max_depth, gamma, subsample+colsample_bytree, 
        'max_depth': 5,  # 构建树的深度，越大越容易过拟合
        'lambda_l2': 1,
#         'num_leaves': 63,
#         'lambda':1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
        'gamma':0.1,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'seed' : 1,
        'scale_pos_weight':1,
        # 'min_child_weight':4,
        # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
        # ，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
        # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
         'verbosity': 0,  # 设置成1则没有运行信息输出，最好是设置为0.
        'eta': eta,  # 如同学习率
        'metric': 'auc'
    }
    bst = lgb.train(param, data_train, num_boost_round=100)
    
#     print("TRAIN")
#     xgb_tra = bst.predict(data_train)  # 训练集
#     weight_lost_cal(Y1_train.reshape(-1, 1), xgb_tra.reshape(-1, 1))
    
    print("TEST：")
    xgb_val = bst.predict(X1_test)  # 交叉验证集
    
    xgb_result += bst.predict(X_test)  # 测试集
#     xgb_result += 1-cur_result  # 测试集

    result += weight_lost_cal(Y1_test.reshape(-1, 1), xgb_val.reshape(-1, 1))

    print("done...")
    
# 计算结果
result /= k
xgb_result /= k


TEST：
AUC value = 0.9785241738828017
Score = 0.4296285463585844
done...
TEST：
AUC value = 0.9789045797174243
Score = 0.4292682926829269
done...
TEST：
AUC value = 0.9772136344246202
Score = 0.4162308385933273
done...
TEST：
AUC value = 0.9785130922089871
Score = 0.41795104261106075
done...
TEST：
AUC value = 0.977036056551142
Score = 0.41778425655976675
done...


In [9]:
print(result)
print(max(xgb_result))
print(min(xgb_result))

0.4221725953611332
0.8617259595966573
0.00021068815700912442


In [10]:
# 输出
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")
data_test['score'] = xgb_result
data_test['score'].round(decimals=5)
data = data_test[['id', 'score']]  #选择表格中的两列
data.to_csv(root + "result_717_1500_V2.csv", index=None, encoding='utf8')